# Running MMS-ASR inference in Colab

In this notebook, we will give an example on how to run simple ASR inference using MMS ASR model.

Credit to epk2112 [(github)](https://github.com/epk2112/fairseq_meta_mms_Google_Colab_implementation)

## Step 1: Clone fairseq-py and install latest version

In [1]:
!mkdir "temp_dir"
!git clone https://github.com/pytorch/fairseq

# Change current working directory
!pwd
%cd "/content/fairseq"
!pip install --editable ./ 
!pip install tensorboardX


Cloning into 'fairseq'...
remote: Enumerating objects: 34665, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 34665 (delta 54), reused 95 (delta 43), pack-reused 34543
Receiving objects: 100% (34665/34665), 24.15 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (25155/25155), done.
/content
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Download MMS model
Un-comment to download your preferred model.
In this example, we use MMS-FL102 for demo purposes.
For better model quality and language coverage, user can use MMS-1B-ALL model instead (but it would require more RAM, so please use Colab-Pro instead of Colab-Free).


In [2]:
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'

# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-05-25 17:15:24--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.59, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4851043301 (4.5G) [binary/octet-stream]
Saving to: ‘./models_new/mms1b_fl102.pt’

mms1b_fl102.pt      100%[===================>]   4.52G   189MB/s    in 26s     

2023-05-25 17:15:51 (176 MB/s) - ‘./models_new/mms1b_fl102.pt’ saved [4851043301/4851043301]



## 3. Prepare audio file
Create a folder on path '/content/audio_samples/' and upload your .wav audio files that you need to transcribe e.g. '/content/audio_samples/audio.wav' 

Note: You need to make sure that the audio data you are using has a sample rate of 16kHz You can easily do this with FFMPEG like the example below that converts .mp3 file to .wav and fixing the audio sample rate

Here, we use a FLEURS english MP3 audio for the example.

In [3]:
!wget -P ./audio_samples/ 'https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3'
!ffmpeg -y -i ./audio_samples/audio.mp3 -ar 16000 ./audio_samples/audio.wav

--2023-05-25 17:15:51--  https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 54.165.66.147, 34.200.186.24, 44.197.252.161, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|54.165.66.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20853 (20K) [audio/mpeg]
Saving to: ‘./audio_samples/audio.mp3’

audio.mp3           100%[===================>]  20.36K  --.-KB/s    in 0.09s   

2023-05-25 17:15:51 (238 KB/s) - ‘./audio_samples/audio.mp3’ saved [20853/20853]

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-fil

# 4: Run Inference and transcribe your audio(s)


In the below example, we will transcribe a sentence in English.

To transcribe other languages: 
1. Go to [MMS README ASR section](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)
2. Open Supported languages link
3. Find your target languages based on Language Name column
4. Copy the corresponding Iso Code
5. Replace `--lang "eng"` with new Iso Code

To improve the transcription quality, user can use language-model (LM) decoding by following this instruction [ASR LM decoding](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)

In [4]:
import os

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" --audio "/content/fairseq/audio_samples/audio.wav"


>>> preparing tmp manifest dir ...
>>> loading model & running inference ...
2023-05-25 17:15:58.935762: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 17:16:01.850306: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Input: /content/fairseq/audio_samples/audio.wav
Output: a tornado is a spinning colum of very low-pressure air which sucks it surrounding air inward and upward
